<a href="https://colab.research.google.com/github/hariszaf/metabolic_toy_model/blob/main/Antony2025/introductionToCOBRApy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction to COBRApy**

### **Prepare your environment**

To run this note, you need COBRApy and gurobi. [See the notebook on preparing your environment for a more detailed explanation](preparingYourEnvironment.ipynb)

In [ ]:
import os
def create_gurobi_license():
    license_content = (
        "# Gurobi WLS license file\n"
        "# Your credentials are private and should not be shared or copied to public repositories.\n"
        "# Visit https://license.gurobi.com/manager/doc/overview for more information.\n"
        "WLSACCESSID=1fedf73b-9471-4da8-bdc7-2aaacf2e30f3\n"
        "WLSSECRET=3bc7d209-a4ec-4195-98be-4b254f181512\n"
        "LICENSEID=940603"
    )
    with open("/content/licenses/gurobi.lic", "w") as f:
        f.write(license_content)
    print("License file created at /content/licenses/gurobi.lic")



# Create directory for the license
os.makedirs("/content/licenses", exist_ok=True)

# Generate the license file
create_gurobi_license()

#add to path
os.environ['GRB_LICENSE_FILE'] = '/content/licenses/gurobi.lic'

In [ ]:
# @title
!pip install gurobipy
!pip install cobra

In [ ]:
import gurobipy
from gurobipy import Model
model = Model("test")
print("Gurobi is working!", "\U0001F600")

#install COBRApy
import cobra
from cobra.io import load_model
model = load_model("textbook")
solution = model.optimize()
print(f"flux balance analysis solution is {solution.objective_value}")
print("COBRApy is working", "\U0001F600")

### **Illustration with three gut species**

![gutSpecies](images/multistabilityPaper.png)

### **Bonus quest**

1) Get the three toy models

In [ ]:
sugar_fermenter = cobra.io.read_sbml_model("files/models/sugar_fermenter.xml")
butyrate_producer = cobra.io.read_sbml_model("files/models/butyrate_producer_toy_model.xml")
acetogen = cobra.io.read_sbml_model("files/models/acetogen_toy_model.xml")

2) Find their exchange reactions

In [ ]:

def findExchanges(model):
    for reaction in model.reactions:
        if 'EX_' in reaction.id: #alternatively use i in model.exchanges
            reac_string = reaction.build_reaction_string(use_metabolite_names =True)
            
            print(f"{reaction.id}\t{reac_string}")


print("Sugar fermenter:")
findExchanges(sugar_fermenter)

print("\nButyrate producer:")
findExchanges(butyrate_producer)

print("\nAcetogen:")
findExchanges(acetogen)

3) Make a medium based on their exchange reactions like a python dictionary

[exchange_reaction] : -10

the `-10` is an arbitrary number representing the maximum amount of the compound that the strain can import from the enviroment.

In [ ]:
def makeMediumFromModels(modelList):
    medium = {}
    
    for model in modelList:
        for reaction in model.exchanges:
            medium[reaction.id] = -10
    
    return medium

medium = makeMediumFromModels([sugar_fermenter, butyrate_producer, acetogen])
print(medium)

4) Make a function to generate a random medium for these exchange reactions

[exchange_reaction] : random lower limit

In [ ]:
import numpy as np

def makeRandomMedium(medium_dict):
    return {reaction: -1*np.random.uniform() for reaction in medium_dict}



med1 = makeRandomMedium(medium)
med2 = makeRandomMedium(medium)
med3 = makeRandomMedium(medium)

print(f"{med1}\n\n{med2}\n\n{med3}")

5) Make a function to apply a medium to a model 

```python
def applyMedium(model, medium_dict):
    #return the growth rate of the model in this medium

```

In [ ]:
def applyMedium(model, medium_dict):
    for reaction in medium_dict:
        if model.reactions.has_id(reaction):
            model.reactions.get_by_id(reaction).lower_limit = medium_dict[reaction]
    
    solution = model.optimize()
    return solution.objective_value

print(f"sugar fermenter: {applyMedium(sugar_fermenter, med1)}")
print(f"\nbutyrate producer: {applyMedium(butyrate_producer, med1)}")
print(f"\nacetogen: {applyMedium(acetogen, med1)}")


6) Apply 1000 random medias to each model and store their growth rates, visualize with an histogram

sf = np.zeros(1000)
bp = np.zeros(1000)
ac = np.zeros(1000)

for i in range(1000):
    medium = makeRandomMedium(medium)
    sf[i] = applyMedium(sugar_fermenter, medium)
    bp[i] = applyMedium(butyrate_producer, medium)
    ac[i] = applyMedium(acetogen, medium)
    
import matplotlib.pyplot as plt

plt.hist(sf, 25, density=True, color = 'red')
plt.show()
plt.hist(bp, 25,  density=True, color = 'blue')
plt.show()
plt.hist(ac, 25,  density=True, color = 'orange')
plt.show()